In [2]:
# ipython -c "%run plot.ipynb"

import pandas as pd
import numpy as np

# Data Preprocessing
# with open("./avg-test-result-in-paper", "r") as f:
with open("./performance-comparison-table-fio", "r") as f:
    df_fio = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')
with open("./performance-comparison-table-real", "r") as f:
    df_real = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')


bar_width = 0.5
true_width = 0.4
num_job = 1

patterns = ['/', '', 'ooo', "///", "\\\\\\", "xxx"]



breakdown_times = [["fp_time", "cmp_time"], ["index_time", "table_time"], ["io_time"], ["fs_time"]]

blks = [1, 512]
blks_fio = ['4K', '2M']

bars = []
global_color_idx = 0
color_map = {}
# grayscale colors
colors = ["black", "dimgray", "lightgray", "gainsboro"]

min_speedup = 100
max_speedup = 0
for df_idx, df in enumerate([df_fio, df_real]):
    for blk_idx, blk in enumerate(blks):
        if df is df_fio:
            real_bs = blks_fio[blk_idx]
            workload_map = {
                0: [],
                25: [],
                50: [],
                75: [],
            }
            workload_kw = "dup_rate"
            workloads = [ 0, 25, 50, 75 ]
            workloads_rename = ["FIO-0%", "FIO-25%", "FIO-50%", "FIO-75%"]
        else:
            real_bs = blk * 4096
            workload_map = {
                "cp": [],
                "homes-2022-fall-50.hitsztrace": [],
                "webmail+online.cs.fiu.edu-110108-113008.1-21.blkparse": [],
                "cheetah.cs.fiu.edu-110108-113008.1.blkparse": [],
            }
            workload_kw = "workload"
            workloads = [ "cp", "homes-2022-fall-50.hitsztrace", "webmail+online.cs.fiu.edu-110108-113008.1-21.blkparse", "cheetah.cs.fiu.edu-110108-113008.1.blkparse" ]
            workloads_rename = ["CP", "OSLab", "Mails", "WebVMs"]
            # workloads = ["write", "webproxy.f"]
            # workloads_remap = ["SW", "WPX"]

            workloads.reverse()
            workloads_rename.reverse()
            
        fss = df['system'].drop_duplicates().reset_index().drop(columns=['index'])['system']
        breakdown_times = [["fp_time", "cmp_time", "index_time"], ["table_time"], ["io_time"], ["fs_time"]]

        print(breakdown_times)
        
        x = list(range(1, len(workloads) + 1))
        
        # fid = df_idx * len(blks) + blk_idx
        # # ax2 = subfig.twinx()
        # pivots = []
        # for fs_idx, fs in enumerate(fss):
        #     for workload_id, workload in enumerate(workloads):
        #         total_time = 0
        #         for idx, time in enumerate(breakdown_times):
        #             width = 0
        #             for t in time:
        #                 width += df[(df[workload_kw] == workload) & (df['system'] == fs) & (df['bs'] == real_bs)][t].iloc[0] / 1000000000
        #             total_time += width
        #         workload_map[workload].append(total_time)
        
        gta_perf = []
        ld_perf = []
        for fs_idx, fs in enumerate(fss):
            perf = gta_perf if fs == "Light-Dedup-J-64bits" else ld_perf
            for workload_id, workload in enumerate(workloads):
                color_idx = 0
                left = 0
                # pivot = x[workload_id] - len(fss) * bar_width / 2 + bar_width / 2 + fs_idx * bar_width 
                # pivots.append(pivot)
                
                inner_left = 0
                inner_left_range = []
                start = False
                for idx, time in enumerate(breakdown_times):
                    width = 0
                    for t in time:
                        width += df[(df[workload_kw] == workload) & (df['system'] == fs) & (df['bs'] == real_bs)][t].iloc[0] / 1000000000 
                    if idx == 1:
                        perf.append(width)
                        # / max(workload_map[workload]) 
        gta_perf = pd.Series(gta_perf)
        ld_perf = pd.Series(ld_perf)
        speedup = (ld_perf - gta_perf) * 100 / ld_perf
        print(speedup.min(), speedup.max())
        if speedup.min() < min_speedup:
            min_speedup = speedup.min()
        if speedup.max() > max_speedup:
            max_speedup = speedup.max()
     
print("GogetaFS speedup range: ", min_speedup, max_speedup)


[['fp_time', 'cmp_time', 'index_time'], ['table_time'], ['io_time'], ['fs_time']]
79.99572684989357 88.86821528285141
[['fp_time', 'cmp_time', 'index_time'], ['table_time'], ['io_time'], ['fs_time']]
92.22403878253697 92.80863679187551
[['fp_time', 'cmp_time', 'index_time'], ['table_time'], ['io_time'], ['fs_time']]
75.35057212635282 83.8749053368349
[['fp_time', 'cmp_time', 'index_time'], ['table_time'], ['io_time'], ['fs_time']]
82.4238514659916 92.19084546576711
